In [ ]:
import os
import json
import re
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
import certifi
from collections import Counter
from tqdm import tqdm
from itertools import islice

# Text processing libraries
try:
    from pyvi import ViTokenizer, ViPosTagger
    PYVI_AVAILABLE = True
except ImportError:
    print("PyVi chưa cài đặt. Cài đặt bằng: pip install pyvi")
    PYVI_AVAILABLE = False

try:
    import spacy
    nlp = spacy.load("vi_core_news_sm")
    SPACY_AVAILABLE = True
except:
    print("spaCy Vietnamese model chưa có. Cài đặt bằng: python -m spacy download vi_core_news_sm")
    SPACY_AVAILABLE = False

# MongoDB connection
MONGO_URI = os.getenv("MONGO_URI")
DB_NAME = "vnexpress_db"
COLLECTION_NAME = "vnexpress_bongda"

class VietnameseTextProcessor:
    def __init__(self):
        # Vietnamese stopwords
        self.stop_words = set([
            'và', 'của', 'trong', 'với', 'là', 'có', 'được', 'cho', 'từ', 'một', 'các',
            'để', 'không', 'sẽ', 'đã', 'về', 'hay', 'theo', 'như', 'cũng', 'này', 'đó',
            'khi', 'những', 'tại', 'sau', 'bị', 'giữa', 'trên', 'dưới', 'ngoài',
            'thì', 'nhưng', 'mà', 'hoặc', 'nếu', 'vì', 'do', 'nên', 'rồi', 'còn', 'đều',
            'chỉ', 'việc', 'người', 'lại', 'đây', 'đấy', 'ở', 'ra', 'vào', 'lên', 'xuống'
        ])
        
        # Football-specific entities
        self.football_teams = [
            'hà nội fc', 'hoàng anh gia lai', 'sài gòn fc', 'than quảng ninh', 'viettel fc',
            'becamex bình dương', 'slna', 'đà nẵng', 'nam định', 'hải phòng fc', 'hcm city',
            'song lam nghệ an', 'quảng nam', 'khánh hòa', 'đội tuyển việt nam', 'tuyển việt nam'
        ]
        
        self.player_patterns = [
            r'cầu thủ\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'tiền đạo\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'thủ môn\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'hậu vệ\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'tiền vệ\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'HLV\s+([A-Za-zÀ-ỹ\s]{3,30})',
            r'huấn luyện viên\s+([A-Za-zÀ-ỹ\s]{3,30})'
        ]
        
        self.competitions = [
            'v-league', 'v.league', 'v league', 'cup quốc gia', 'aff cup', 'sea games',
            'world cup', 'asian cup', 'champions league', 'afc cup', 'u23', 'u22', 'u19'
        ]
    
    def clean_text(self, text):
        """Làm sạch và chuẩn hóa text tiếng Việt"""
        if not text:
            return ""
        
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\sàáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđĐ]', ' ', text)
        text = text.lower()
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize_vietnamese(self, text):
        """Tách từ tiếng Việt"""
        if PYVI_AVAILABLE:
            try:
                return ViTokenizer.tokenize(text).split()
            except:
                pass
        return text.split()
    
    def remove_stopwords(self, tokens):
        return [token for token in tokens if token not in self.stop_words and len(token) > 1]
    
    def extract_entities(self, text):
        entities = {'teams': [], 'players': [], 'competitions': []}
        text_lower = text.lower()
        
        for team in self.football_teams:
            if team in text_lower:
                entities['teams'].append(team)
        
        for pattern in self.player_patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                player_name = match.group(1).strip()
                if len(player_name) > 3:
                    entities['players'].append(player_name)
        
        for comp in self.competitions:
            if comp in text_lower:
                entities['competitions'].append(comp)
        
        return entities
    
    def process_document(self, doc):
        title = doc.get('title', '')
        content = doc.get('content', '')
        full_text = f"{title} {content}"
        
        cleaned_text = self.clean_text(full_text)
        tokens = self.tokenize_vietnamese(cleaned_text)
        filtered_tokens = self.remove_stopwords(tokens)
        entities = self.extract_entities(full_text)
        
        return {
            'doc_id': str(doc.get('_id', '')),
            'title': title,
            'url': doc.get('url', ''),
            'date': doc.get('date', ''),
            'author': doc.get('author', ''),
            'original_length': len(full_text),
            'cleaned_text': cleaned_text,
            'tokens': tokens,
            'filtered_tokens': filtered_tokens,
            'token_count': len(tokens),
            'filtered_count': len(filtered_tokens),
            'entities': entities,
            'entity_count': sum(len(entities[key]) for key in entities)
        }

def connect_to_database():
    try:
        client = MongoClient(MONGO_URI, tls=True, tlsCAFile=certifi.where())
        db = client[DB_NAME]
        collection = db[COLLECTION_NAME]
        doc_count = collection.count_documents({})
        print(f"Kết nối database thành công!")
        print(f"Tổng số documents: {doc_count}")
        return client, db, collection
    except Exception as e:
        print(f"Lỗi kết nối database: {e}")
        return None, None, None

def process_all_documents(collection, limit=None):
    processor = VietnameseTextProcessor()
    if limit:
        documents = collection.find().limit(limit)
        total_docs = limit
    else:
        documents = collection.find()
        total_docs = collection.count_documents({})
    
    print(f"Bắt đầu xử lý {total_docs} documents...")
    processed_results, error_count = [], 0
    
    for doc in tqdm(documents, desc="Processing documents", total=total_docs):
        try:
            result = processor.process_document(doc)
            processed_results.append(result)
        except Exception as e:
            error_count += 1
            print(f"Lỗi xử lý document {doc.get('_id', 'unknown')}: {e}")
    
    print(f"Hoàn thành! Xử lý thành công: {len(processed_results)}, Lỗi: {error_count}")
    return processed_results

def analyze_results(results):
    print("\nPHÂN TÍCH KẾT QUẢ:")
    print(f"Tổng số documents đã xử lý: {len(results)}")
    
    token_counts = [r['token_count'] for r in results]
    filtered_counts = [r['filtered_count'] for r in results]
    entity_counts = [r['entity_count'] for r in results]
    
    print(f"\nThống kê Tokens:")
    print(f"  - Trung bình tokens/document: {sum(token_counts)/len(token_counts):.1f}")
    print(f"  - Trung bình filtered tokens/document: {sum(filtered_counts)/len(filtered_counts):.1f}")
    print(f"  - Trung bình entities/document: {sum(entity_counts)/len(entity_counts):.1f}")
    
    all_teams, all_players, all_competitions = [], [], []
    for r in results:
        all_teams.extend(r['entities']['teams'])
        all_players.extend(r['entities']['players'])
        all_competitions.extend(r['entities']['competitions'])
    
    print(f"\nTop 10 Đội bóng được nhắc đến nhiều nhất:")
    for team, count in Counter(all_teams).most_common(10):
        print(f"  - {team}: {count} lần")
    
    print(f"\nTop 10 Cầu thủ được nhắc đến nhiều nhất:")
    for player, count in Counter(all_players).most_common(10):
        print(f"  - {player}: {count} lần")
    
    print(f"\nTop 10 Giải đấu được nhắc đến nhiều nhất:")
    for comp, count in Counter(all_competitions).most_common(10):
        print(f"  - {comp}: {count} lần")

def save_results(results, filename="processed_data.json"):
    filepath = os.path.join("d:\\data\\Search_Engine", filename)
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2, default=str)
    print(f"Đã lưu kết quả vào {filepath}")

def analyze_multiple_documents(collection, processor, n=50, top_n=50, filename=None):
    """Lấy ngẫu nhiên n documents, gộp tokens và tính tần suất chung"""
    docs = collection.aggregate([{"$sample": {"size": n}}])
    all_tokens = []
    
    for idx, doc in enumerate(docs, 1):
        result = processor.process_document(doc)
        tokens = result["filtered_tokens"]
        all_tokens.extend(tokens)   # gộp tất cả tokens lại
    
    # Tính tần suất từ toàn bộ tokens
    counter = Counter(all_tokens)
    total = sum(counter.values())
    
    data = []
    for i, (word, freq) in enumerate(counter.most_common(top_n), 1):
        Pr = freq / total * 100
        data.append({
            "Word": word,
            "Freq": freq,
            "r": i,
            "Pr(%)": round(Pr, 2),
            "r*Pr": round(i * Pr / 100, 3)
        })
    
    df = pd.DataFrame(data)
    print(df.to_string(index=False))
    
    # Xuất file CSV 1 lần
    if not filename:
        filename = f"wordfreq_{n}docs_total_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\nĐã lưu kết quả {n} documents (gộp) vào: {filename}")
    
    return df

def generate_ngrams(tokens, n):
    """Sinh n-grams từ list tokens"""
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def analyze_and_save_ngrams(collection, processor, n_docs=1800, max_n=5, top_k=50, filename=None):
    """
    Phân tích n-grams (1 -> max_n), lưu tất cả vào 1 file CSV duy nhất
    """
    docs = collection.aggregate([{"$sample": {"size": n_docs}}])

    all_tokens_list = []
    for doc in docs:
        result = processor.process_document(doc)
        all_tokens_list.append(result["filtered_tokens"])

    all_data = []

    for n in range(1, max_n+1):
        ngrams_all = []
        for tokens in all_tokens_list:
            ngrams_all.extend(generate_ngrams(tokens, n))
        
        counter = Counter(ngrams_all)
        for phrase, freq in counter.most_common(top_k):
            all_data.append({
                "n": n,
                "Frequency": freq,
                "Phrase": phrase
            })
    
    df = pd.DataFrame(all_data)

    if not filename:
        filename = f"ngrams_{n_docs}docs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"Đã lưu kết quả n-grams (1-{max_n}) vào: {filename}")
    
    return df




def main():
    print("BẮT ĐẦU XỬ LÝ TEXT TỪ MONGODB")
    print("=" * 50)
    
    client, db, collection = connect_to_database()
    if collection is None:
        print("Không thể kết nối database!")
        return
    
    results = process_all_documents(collection, limit=5)  
    if results:
        analyze_results(results)
        save_results(results, f"processed_vnexpress_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
    
    # Thử lấy 1 document ngẫu nhiên và phân tích từ vựng
    processor = VietnameseTextProcessor()
    analyze_multiple_documents(collection, processor, n=50, top_n=50)
    analyze_and_save_ngrams(collection, processor, n_docs=1830, max_n=5, top_k=100)

    client.close()
    print("\nĐã đóng kết nối database")
    print("HOÀN THÀNH!")

if __name__ == "__main__":
    main()


spaCy Vietnamese model chưa có. Cài đặt bằng: python -m spacy download vi_core_news_sm
BẮT ĐẦU XỬ LÝ TEXT TỪ MONGODB
Kết nối database thành công!
Tổng số documents: 1832
Bắt đầu xử lý 5 documents...


Processing documents: 100%|██████████| 5/5 [00:00<00:00, 21.28it/s]


Hoàn thành! Xử lý thành công: 5, Lỗi: 0

PHÂN TÍCH KẾT QUẢ:
Tổng số documents đã xử lý: 5

Thống kê Tokens:
  - Trung bình tokens/document: 727.4
  - Trung bình filtered tokens/document: 541.4
  - Trung bình entities/document: 19.2

Top 10 Đội bóng được nhắc đến nhiều nhất:
  - nam định: 2 lần
  - tuyển việt nam: 2 lần
  - hà nội fc: 1 lần
  - đà nẵng: 1 lần
  - đội tuyển việt nam: 1 lần

Top 10 Cầu thủ được nhắc đến nhiều nhất:
  - Kim Sang: 3 lần
  - Park Hang: 2 lần
  - trẻ cần được thi đấu thường xu: 1 lần
  - nhập tịch vì sử dụng giấy tờ g: 1 lần
  - vào sân: 1 lần
  - bị phạt: 1 lần
  - của Quy định này có đoạn: 1 lần
  - phải được đăng ký đúng quy địn: 1 lần
  - được coi là đủ tư cách thi đấu: 1 lần
  - hợp lệ: 1 lần

Top 10 Giải đấu được nhắc đến nhiều nhất:
  - u23: 4 lần
  - world cup: 3 lần
  - asian cup: 3 lần
  - v-league: 2 lần
  - champions league: 1 lần
  - aff cup: 1 lần
Đã lưu kết quả vào d:\data\Search_Engine\processed_vnexpress_20251004_180550.json
    Word  Freq  r